### Exercise 1.1:
Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented.

In [1]:
10

10

In [2]:
(+ 5 3 4)

12

In [3]:
(- 9 1)

8

In [4]:
(/ 6 2)

3

In [5]:
(+ (* 2 4) (- 4 6))

6

In [6]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))

19

In [7]:
(= a b)

#f

In [8]:
(if (and (> b a) (< b (* a b)))
  b
  a)

4

In [9]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

In [10]:
(+ 2 (if (> b a) b a))

6

In [11]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

### Exercise 1.2:
Translate the following expression into prefix form:

$$\frac{5 + 4 + (2 - (3 - (6 + \frac{4}{5})))}{3(6 - 2)(2 - 7)}$$

In [12]:
(/ (+ 5 4 (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))

-37/150

### Exercise 1.3:

Define a procedure that takes three numbers
as arguments and returns the sum of the squares of the two
larger numbers.

In [13]:
(define (square x) (* x x))
(define (sum-of-square x y) (+ (square x) (square y)))

(define (sum-of-square-of-two-larger-numbers a b c)
  (if (or (> a b) (> a c))
      (sum-of-square a (if (> b c) b c))
      (sum-of-square b c)))

(sum-of-square-of-two-larger-numbers 3 1 2)


13

### Exercise 1.4:

Observe that our model of evaluation allows for combinations whose operators are compound expressions. Use this observation to describe the behavior of the following procedure:

```scheme
(define (a-plus-abs-b a b)
  ((if (> b 0) + -) a b))
```

### Answer
The result of the expression `(if (> b 0) + -)` is a built-in procedure (`+` or `-` depending on the value of `b`). That procedure is substituted in place of the `if` expression. For example: `(+ a b)` or as `(- a b)`. This expression is then evaluated to return the expected result. As the name of the procedure suggests, it adds `a` with the absolute value of `b`.

### Exercise 1.5:
Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative-order evaluation or normal-order evaluation. He defines the following two procedures:

```scheme
(define (p) (p))
(define (test x y)
  (if (= x 0) 0 y))
```

Ten he evaluates the expression
```scheme
(test 0 (p))
```

What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form if is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)

### Answer:
**Applicative-Order**: The interpreter evaluates the operator and operands and then applies the resulting procedure to the resulting arguments.

**Normal-Order**: First substitute operand expressions for parameters until it obtained an expression involving only primitive operators.

**Note**: Lisp uses applicative-order, partly because of the additional efficiency but also it is complicated to deal with normal-order evaluation when used outside simple procedures. 

#### Behaviour under applicative-order evaluation:
Before invoking `test` procedure, all of its arguments will be evaluated, `0` will evaluate to `0`, `(p)` will invoke itself, resulting in an loop that will never terminate.

#### Behaviour under normal-order evaluation:
The body of the `test` and `p` procedure will be substituted in the evaluating expression. Since the procedure `p` cannot be substituted further, the invocation will be reduced to the following:

```scheme
(if (= 0 0) 0 (p))
```


 In this evaluation, first the predicate of the `if` block is evaluated. Since `(= 0 0)` is true, only the true block is evaluated (and not the else block). The function will terminate with value as `0`.


### Exercise 1.6:
Alyssa P. Hacker doesn’t see why if needs to be provided as a special form. “Why can’t I just define it as an ordinary procedure in terms of cond?” she asks. Alyssa’s
friend Eva Lu Ator claims this can indeed be done, and she defines a new version of if:

```scheme
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))
```
Eva demonstrates the program for Alyssa:
```scheme
(new-if (= 2 3) 0 5)
5
(new-if (= 1 1) 0 5)
0
```
Delighted, Alyssa uses new-if to rewrite the square-root
program:
```scheme
(define (sqrt-iter guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter (improve guess x) x)))
```
What happens when Alyssa attempts to use this to compute square roots? Explain.

### Answer:
When Alyssa attempts to use this to compute square roots, the function will not terminate. Because, the `new-if` will be a procedure with applicative-order execution. the `predicate`, `then-clause` and `else-clause` will always be executed, resulting in an infinite loop.

### Exercise 1.7: 
The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how guess changes from one iteration to the next and to stop when the change is a very small fraction of the guess. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers?

### Answer
- When used with small numbers the result will not be accurate because the `good-enough?` procedure will square the guess and it would result in a bigger jump than `0.0001`.

For example, in the procedure below, the square of square root of `0.0000005` returns `0.0000006`

In [14]:
(define (sqrt x)
  (define (good-enough? guess x) (< (abs (- (* guess guess) x)) 0.0001))
  (define (average x y) (/ (+ x y) 2))
  (define (improve guess x) (average guess (/ x guess)))

  (define (sqrt-iter guess x)
    (if (good-enough? guess x) guess (sqrt-iter (improve guess x) x)))
  (sqrt-iter 1.0 x))
  
(define (square x) (* x x))
(square (sqrt 0.0000005))

6.136874195037108e-5

- When used with larger numbers, the execution will not terminate because when `x` is large, the difference between first `guess` and the subsequent `guess` will be too high for the precision to reach `0.0001` - resulting in infinite oscillation.

#### Solution:

In [15]:
(define (sqrt2 x)
  (define (good-enough? guess new-guess) (< (abs (- guess new-guess)) 0.0001))
  (define (average x y) (/ (+ x y) 2))
  (define (improve guess x) (average guess (/ x guess)))
  (define (sqrt-iter guess x)
    (if (good-enough? guess (improve guess x)) (improve guess x) (sqrt-iter (improve guess x) x)))

  (sqrt-iter 1.0 x))

(square (sqrt2 100000000000000000))

100000000000000020.0

In [16]:
(square (sqrt2 0.0000005))

5.061373437048838e-7

### Exercise 1.8:
Newton’s method for cube roots is based on the fact that if y is an approximation to the cube root of x, then a beter approximation is given by the value

$$\frac{\frac{x}{y^{2}} + 2y}{3}$$

Use this formula to implement a cube-root procedure analogous to the square-root procedure.

### Answer

In [17]:
(define (cubrt x)
  (define (good-enough? guess x) (< (abs (- (* guess guess guess) x)) 0.0001))
  (define (improve guess x) (/ (+ (/ x (* guess guess)) (* 2 guess)) 3))
  (define (cubrt-iter guess x)
    (if (good-enough? guess x) guess (cubrt-iter (improve guess x) x)))

  (cubrt-iter 1.0 x))

(cubrt (* 9 9 9))

9.000000000053902